In [1]:
data_var = '2024-01-29'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
11160,2024-01-29,Eua Ncaa,23:00,Prairie View A&M,Alabama A&M,1.30,3.66,146.5,1.80,1.95,-8.5,2.00,1.71,hbDg9Clg,0.769231,0.273224,0.555556,0.512821,0.042455,158.410,65.881191,0.415890,1.2,1.643168,1.369306,113.23,2.318,0.982482,0.423849,-20.0,175.040,71.909461,0.410817,0.6,1.341641,2.236068,289.11,2.634,1.125935,0.427462,-36.0,67,69,1.69,4.19,0.000,338.290,0.672892,0.056569,0.110545,0.00,0.000,inf,0.000000,0.0,0.000000,-1.37,-0.274,-9.708029,0.233918,0.2,-0.033918
11161,2024-01-29,Eua Ncaa,23:00,Bethune Cookman,Alcorn State,1.63,2.37,152.5,1.91,1.91,-3.5,2.00,1.71,pppXmdAT,0.613497,0.421941,0.523560,0.523560,0.035438,162.676,57.132671,0.351205,1.8,1.643168,0.912871,202.54,2.090,0.624460,0.298785,15.0,133.720,15.116104,0.113043,1.8,1.643168,0.912871,131.48,1.934,0.361151,0.186738,11.0,82,76,2.47,1.73,180.760,286.780,0.261630,0.000000,0.110545,3.84,0.768,0.820312,0.000000,0.0,0.000000,-1.36,-0.272,-5.036765,0.221267,0.2,-0.021267
11162,2024-01-29,Eua Ncaa,22:30,Texas Southern,Alabama State,1.58,2.47,132.5,1.80,1.95,-5.5,2.10,1.67,hIVWXBRk,0.632911,0.404858,0.555556,0.512821,0.037770,121.298,37.504001,0.309189,1.8,1.643168,0.912871,106.25,1.658,0.706944,0.426384,47.0,122.358,31.078830,0.253999,2.4,1.341641,0.559017,164.28,1.758,0.327674,0.186390,27.0,85,74,1.25,2.22,138.186,586.090,0.310778,0.056569,0.161303,-0.62,-0.124,-4.677419,0.000000,0.0,0.000000,0.03,0.006,245.000000,0.360781,0.5,0.139219
11163,2024-01-29,Eua Ncaa,22:30,Southern,Miss. Valley St.,1.03,17.00,132.5,1.91,1.91,-20.5,2.00,1.71,jypJOqQ2,0.970874,0.058824,0.523560,0.523560,0.029697,109.274,8.813647,0.080656,1.8,1.643168,0.912871,98.40,1.472,0.252923,0.171823,25.0,563.092,294.529996,0.523058,0.0,0.000000,NaN,690.00,9.836,5.666077,0.576055,-78.0,80,46,1.23,15.00,119.778,656.000,1.252634,0.000000,0.110545,2.55,0.510,0.058824,0.000000,0.0,0.000000,-5.00,-1.000,-16.000000,0.188185,0.0,-0.188185
11164,2024-01-29,Eua Ncaa,22:30,Grambling St.,Arkansas-Pine Bluff,1.37,3.29,145.5,1.91,1.91,-7.5,2.00,1.71,6BEM68XS,0.729927,0.303951,0.523560,0.523560,0.033878,117.816,42.271662,0.358794,2.4,1.341641,0.559017,56.16,1.754,0.405130,0.230975,46.0,202.298,76.854670,0.379908,1.8,1.643168,0.912871,293.04,2.780,1.347127,0.484578,-16.0,54,66,1.04,4.44,160.598,565.260,0.582680,0.000000,0.110545,-0.37,-0.074,-5.000000,0.000000,0.0,0.000000,-1.03,-0.206,-11.116505,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11222,2024-01-29,Cazaquistão Liga Nacional Feminina,06:00,Tomiris Almaty F,Astana F,5.13,1.14,131.5,1.83,1.83,11.5,1.83,1.83,zcH0qTKr,0.194932,0.877193,0.546448,0.546448,0.072125,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,70,69,1.65,1.04,0.000,0.000,0.899954,0.000000,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11223,2024-01-29,Cazaquistão Higher League,03:30,Karaganda,Bars Petropavlovsk,12.60,1.02,137.5,1.85,1.85,19.5,2.00,1.72,KlD8m3b7,0.079365,0.980392,0.540541,0.540541,0.059757,310.544,221.979521,0.7

## Apostas do dia

### Back Home V3 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
11162,22:30,Eua Ncaa,Texas Southern,Alabama State,1.58,Back Home
11166,22:00,Eua Ncaa,Md.-East. Shore,South Carolina St,1.83,Back Home
11198,21:30,Eua Nba,Brooklyn Nets,Utah Jazz,1.93,Back Home
11208,21:00,Argentina Liga A,Argentino,Zarate,2.18,Back Home
